In [144]:
import pandas as pd
df = pd.read_csv('/Users/temirlanurazgaliev/Downloads/testovoe.csv')

In [146]:
df

,event_category,event_action,event_label,total_events,unique_events
0,city_landing,price_button_submit,Tula / Показать предложения (11),8,7
1,city_landing,price_button_submit,Sochi / Показать предложения (39),12,10
2,city_landing,search-tools-button_open,Penza / Сортировка,2,2
3,city_landing,filters-categories_click,Gelendzhik / ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ,1,1
4,city_landing,price_button_submit,Moscow / Показать предложения (345),2,2
...,...,...,...,...,...
19087,city_landing,search-tools-button_open,Istanbul / ЭКСКУРСИИ В СТАМБУЛЕ,38,32
19088,city_landing,filters-categories_click,Ufa / Пешеходные,3,3
19089,city_landing,search-tools-button_open,Yerevan / Фильтры Выбрано: 0,16,16
19090,city_landing,price_button_submit,Pskov / Показать предложения (35),7,7


In [148]:
df[['city', 'event_on_site']] = df['event_label'].str.split(' / ', expand=True)  # Создаю 2 колонки из колонки event_label

In [150]:
df

,event_category,event_action,event_label,total_events,unique_events,city,event_on_site
0,city_landing,price_button_submit,Tula / Показать предложения (11),8,7,Tula,Показать предложения (11)
1,city_landing,price_button_submit,Sochi / Показать предложения (39),12,10,Sochi,Показать предложения (39)
2,city_landing,search-tools-button_open,Penza / Сортировка,2,2,Penza,Сортировка
3,city_landing,filters-categories_click,Gelendzhik / ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ,1,1,Gelendzhik,ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ
4,city_landing,price_button_submit,Moscow / Показать предложения (345),2,2,Moscow,Показать предложения (345)
...,...,...,...,...,...,...,...
19087,city_landing,search-tools-button_open,Istanbul / ЭКСКУРСИИ В СТАМБУЛЕ,38,32,Istanbul,ЭКСКУРСИИ В СТАМБУЛЕ
19088,city_landing,filters-categories_click,Ufa / Пешеходные,3,3,Ufa,Пешеходные
19089,city_landing,search-tools-button_open,Yerevan / Фильтры Выбрано: 0,16,16,Yerevan,Фильтры Выбрано: 0
19090,city_landing,price_button_submit,Pskov / Показать предложения (35),7,7,Pskov,Показать предложения (35)


In [308]:
df.isna().sum()  # проверим на пустые значения

event_category       0
event_action         0
event_label          0
total_events         0
unique_events        0
city                 0
event_on_site     2079
dtype: int64

In [314]:
df.duplicated().sum()  # проверим на дубликаты

0

Пустых значений и дубликатов нет - супер, идем дальше

1) Сколько людей пользуются фильтрами?

In [318]:
#  есть ситуации, в которых один и тот же пользователь может посетить страницы разных городов. Например, Пермь и Сан-Франциско. В двух этих событиях этот пользователь будет считаться как уникальный, что может помешать корректно ответить на вопрос о количестве уникальных пользователей, которые используют фильтры! Но так как инфа дана по уже сагрегированным таблицам, то ответ могу дать только такой 

In [320]:
unique_users = df.query("event_action == 'price_button_submit'") \
                   .aggregate({'unique_events':'sum'})

In [322]:
unique_users

unique_events    63063
dtype: int64

Таким образом узнали, что фильтрами пользуется 63063 человек. Мне стало интересно посмотреть конверсии. Поэтому я решил покопаться чуть глубже

In [325]:
df.groupby(['event_category', 'event_action'], as_index=False) \
    .agg(total_events=('total_events', 'sum'), 
         unique_events=('unique_events', 'sum')) \
    .sort_values('total_events', ascending=False)

,event_category,event_action,total_events,unique_events
0,City Page,Page Visit,2124993,1384478
11,city_landing,search-tools-button_open,94210,85773
4,city_landing,filters-categories_click,80065,62539
7,city_landing,price_button_submit,71890,63063
13,city_landing,ticket-type_checkbox,22882,17918
12,city_landing,start_date_click,19939,14501
3,city_landing,end_date_click,14638,11738
6,city_landing,price_button_close,14247,11646
2,city_landing,dates_filter_mobile,13038,10272
8,city_landing,price_first,8572,6676


Таким образом, узнал, что страницы городов(Page Visit) посетило 1384478 уникальных пользователей. Если я парвильно понял логику мобилки, то 85773 пользователя открыли настройки фильтров, сортировок и т.д.(search-tools-button_open) Но лишь 63063 пользователей нажали на кнопку применения этих фильтров(price_button_submit).

In [328]:
round(85773 / 1384478 * 100, 2)

6.2

Такой процент от общего числа уникальных пользовавтелей зашел в настройки фильтров, сортировок и т.д.

In [331]:
round(63063 / 85773 * 100, 2)

73.52

Но лишь 73.52 процентов пользователей зашедших в настройки фильтров и т.д. применили эти настройки для поиска интересующих их экскурсий.
О чем это может говорить? У меня есть две гипотезы: либо пользователи не находят в доступных фильтрах и сортировках того, что искали, либо пользователю непонятно как их использовать

2) В каких городах фильтрами пользуются больше? В каких меньше?

In [335]:
events_per_city = (df.query("event_action == 'price_button_submit'")
                   .groupby('city', as_index = False)
                   .aggregate({'unique_events':'sum'})
                   .sort_values('unique_events', ascending = False))

In [337]:
events_per_city

,city,unique_events
132,Kaliningrad,5222
262,Saint Petersburg,5028
197,Moscow,3176
286,Sochi,2909
124,Istanbul,2357
...,...,...
317,Tuscany,1
183,Marbella,1
67,Chemal,1
187,Medellin,1


In [339]:
events_per_city.head(10)

,city,unique_events
132,Kaliningrad,5222
262,Saint Petersburg,5028
197,Moscow,3176
286,Sochi,2909
124,Istanbul,2357
81,Dubai,2095
53,Budapest,1843
139,Kazan,1839
13,Anapa,1619
278,Sharm-el-Sheikh,1440


ТОП-10 городов по использованию фильтров среди пользователей. Можно обратить внимание, что среди этих 10 городов, 6 городов - это города России, а 4 - зарубежные направления.

In [342]:
events_per_city['less_than_10'] = events_per_city['unique_events'] <= 10

In [344]:
events_per_city[events_per_city['less_than_10'] == True]

,city,unique_events,less_than_10
128,Izhevsk,10,True
88,Elista,10,True
210,Nice,10,True
5,Akko,10,True
137,Kaspiysk,10,True
...,...,...,...
317,Tuscany,1,True
183,Marbella,1,True
67,Chemal,1,True
187,Medellin,1,True


Так как во многих городах число пользователей, которые используют фильтры, равно 1, то я посчитал более полезным посмотреть общее количество городов, в которых число таковых пользователей меньше или равно 10. Таких городов получилось 178(из них есть один под названием service-city, скорее всего тестовая строка). Большинство этих городов находится за пределами России, но есть и города в РФ пользователи, которых пока мало используют фильтры.

3) Какие разделы фильтров наиболее востребованы? Фильтры, сортировки, категории?

Смотрим, все открытия инстурментов фильтрации, сортировки и категорий

In [349]:
data_filters = df[df['event_action'] == 'search-tools-button_open'].reset_index(drop=True).copy()

In [351]:
data_filters

,event_category,event_action,event_label,total_events,unique_events,city,event_on_site
0,city_landing,search-tools-button_open,Penza / Сортировка,2,2,Penza,Сортировка
1,city_landing,search-tools-button_open,Crete / Фильтры Выбрано: 1,9,9,Crete,Фильтры Выбрано: 1
2,city_landing,search-tools-button_open,Irkutsk / КБЖД,2,2,Irkutsk,КБЖД
3,city_landing,search-tools-button_open,Adler / КАКИЕ ЭКСКУРСИИ В АДЛЕРЕ,1,1,Adler,КАКИЕ ЭКСКУРСИИ В АДЛЕРЕ
4,city_landing,search-tools-button_open,Haifa / Категории Все,4,4,Haifa,Категории Все
...,...,...,...,...,...,...,...
4843,city_landing,search-tools-button_open,Tbilisi / Категории Обзорные,1,1,Tbilisi,Категории Обзорные
4844,city_landing,search-tools-button_open,Alanya / Морские прогулки,6,5,Alanya,Морские прогулки
4845,city_landing,search-tools-button_open,Anapa / В Крым,5,4,Anapa,В Крым
4846,city_landing,search-tools-button_open,Istanbul / ЭКСКУРСИИ В СТАМБУЛЕ,38,32,Istanbul,ЭКСКУРСИИ В СТАМБУЛЕ


Посмотрим все ункальные записи в колонке 'event_on_site'.

In [359]:
data_filters['event_on_site'].value_counts()[15:45]

event_on_site
Групповые                              60
Популярные                             55
Выбрано: 4                             55
Индивидуальные                         52
Обзорные                               50
Необычные                              49
Автобусные                             44
Фильтры Выбрано: 2                     42
Для детей                              39
Вечерние экскурсии                     36
Выбрано: 5                             35
Пешеходные                             33
Гастрономические                       31
Сортировка Сначала самые дешевые       28
Фильтры Выбрано: 3                     26
Ночные экскурсии                       21
Выездные                               20
Сортировка Сначала самые популярные    18
Категории Необычные                    13
Фильтры Выбрано: 4                     13
Морские прогулки                       13
Трансферы                              11
Selected: 0                            11
Категории Автобусные

Если внимательно посмотреть эти записи, можно найти информацию, записанную на латинице.

Создадим список со всеми значениями 'event_on_site'

In [363]:
data = data_filters.groupby(['event_on_site'], as_index=False) \
    .agg(total_events=('total_events', 'sum'), 
         unique_events=('unique_events', 'sum')) \
    .sort_values('total_events', ascending=False)

distinct_data = list(data['event_on_site'])
distinct_data

['Фильтры',
 'Выбрано: 0',
 'По рейтингу',
 'Категории',
 'Все',
 'Сортировка',
 'Выбрано: 1',
 'Выбрано: 2',
 'Фильтры Выбрано: 0',
 'Выбрано: 3',
 'Категории Все',
 'Сортировка По рейтингу',
 'Сначала самые дешевые',
 'Фильтры Выбрано: 1',
 'Сначала самые популярные',
 'Выбрано: 4',
 'Фильтры Выбрано: 2',
 'Популярные',
 'ЭКСКУРСИИ КАЛИНИНГРАД',
 'Необычные',
 'Групповые',
 'Обзорные',
 'Индивидуальные',
 'Автобусные',
 'Выбрано: 5',
 'Гастрономические',
 'Пешеходные',
 'Для детей',
 'Морские прогулки',
 'Вечерние экскурсии',
 'ЭКСКУРСИИ ИЗ НАЛЬЧИКА',
 'ЭКСКУРСИИ ИЗ КИСЛОВОДСКА',
 'Ночные экскурсии',
 'Фильтры Выбрано: 3',
 'Сортировка Сначала самые дешевые',
 'Выездные',
 'ЭКСКУРСИИ ПЯТИГОРСК',
 'ЭКСКУРСИИ В КАЛИНИНГРАДЕ',
 'ЭКСКУРСИИ В СТАМБУЛЕ',
 'По пригородам',
 'ЭКСКУРСИИ В АНАПЕ',
 'ЭКСКУРСИИ ПСКОВ',
 'ЭКСКУРСИИ ПО СТАМБУЛУ',
 'ЭКСКУРСИИ КРИТ',
 'ЭКСКУРСИИ ЕКАТЕРИНБУРГ',
 'СЕВАСТОПОЛЬ ЭКСКУРСИИ',
 'ЭКСКУРСИИ СТАМБУЛ',
 'ЭКСКУРСИИ ИЗ ЕССЕНТУКОВ',
 'ЭКСКУРСИИ ПО КАЗАНИ',
 'Развл

Далее значения моего списка distinct_data я делаю ключами в словаре event_dict. В качестве значений я задам им один из 3 разделов: 'Фильтры', 'Сортировка', 'Категории'. Можно, конечно, в ручную произвести поиск триггерных слов для разделения на разделы, но эффективнее дать эту работу ИИ, поэтому я просто загрузил список, и ИИ уже сам распределил, что и к чему относится. Так как 'Категории' самый обширный по количеству ключей раздел, его ключи я не перечислял, а просто оставил в else.

In [366]:
filter_triggers = ['фильтр', 'выбрано', 'filter', 'selected', 'seleccionado', 'ausgewählt']
sort_triggers = ['сорт', 'по рейтингу', 'сначала самые', 'most popular first', 'by rating', 'sort']

event_dict = {
    key: (
        'Фильтры' if any(i in key.lower() for i in filter_triggers)
        else 
        'Сортировка' if any(i in key.lower() for i in sort_triggers)
        else 'Категории'
    ) for key in distinct_data}

event_dict

{'Фильтры': 'Фильтры',
 'Выбрано: 0': 'Фильтры',
 'По рейтингу': 'Сортировка',
 'Категории': 'Категории',
 'Все': 'Категории',
 'Сортировка': 'Сортировка',
 'Выбрано: 1': 'Фильтры',
 'Выбрано: 2': 'Фильтры',
 'Фильтры Выбрано: 0': 'Фильтры',
 'Выбрано: 3': 'Фильтры',
 'Категории Все': 'Категории',
 'Сортировка По рейтингу': 'Сортировка',
 'Сначала самые дешевые': 'Сортировка',
 'Фильтры Выбрано: 1': 'Фильтры',
 'Сначала самые популярные': 'Сортировка',
 'Выбрано: 4': 'Фильтры',
 'Фильтры Выбрано: 2': 'Фильтры',
 'Популярные': 'Категории',
 'ЭКСКУРСИИ КАЛИНИНГРАД': 'Категории',
 'Необычные': 'Категории',
 'Групповые': 'Категории',
 'Обзорные': 'Категории',
 'Индивидуальные': 'Категории',
 'Автобусные': 'Категории',
 'Выбрано: 5': 'Фильтры',
 'Гастрономические': 'Категории',
 'Пешеходные': 'Категории',
 'Для детей': 'Категории',
 'Морские прогулки': 'Категории',
 'Вечерние экскурсии': 'Категории',
 'ЭКСКУРСИИ ИЗ НАЛЬЧИКА': 'Категории',
 'ЭКСКУРСИИ ИЗ КИСЛОВОДСКА': 'Категории',
 'Ночные э

In [368]:
data_filters['sections'] = data_filters['event_on_site'].apply(lambda x: event_dict[x])
data_filters

,event_category,event_action,event_label,total_events,unique_events,city,event_on_site,sections
0,city_landing,search-tools-button_open,Penza / Сортировка,2,2,Penza,Сортировка,Сортировка
1,city_landing,search-tools-button_open,Crete / Фильтры Выбрано: 1,9,9,Crete,Фильтры Выбрано: 1,Фильтры
2,city_landing,search-tools-button_open,Irkutsk / КБЖД,2,2,Irkutsk,КБЖД,Категории
3,city_landing,search-tools-button_open,Adler / КАКИЕ ЭКСКУРСИИ В АДЛЕРЕ,1,1,Adler,КАКИЕ ЭКСКУРСИИ В АДЛЕРЕ,Категории
4,city_landing,search-tools-button_open,Haifa / Категории Все,4,4,Haifa,Категории Все,Категории
...,...,...,...,...,...,...,...,...
4843,city_landing,search-tools-button_open,Tbilisi / Категории Обзорные,1,1,Tbilisi,Категории Обзорные,Категории
4844,city_landing,search-tools-button_open,Alanya / Морские прогулки,6,5,Alanya,Морские прогулки,Категории
4845,city_landing,search-tools-button_open,Anapa / В Крым,5,4,Anapa,В Крым,Категории
4846,city_landing,search-tools-button_open,Istanbul / ЭКСКУРСИИ В СТАМБУЛЕ,38,32,Istanbul,ЭКСКУРСИИ В СТАМБУЛЕ,Категории


In [370]:
FilterOrSortOrCategory = data_filters.groupby('sections', as_index = False) \
.aggregate({'unique_events':'sum'})

In [372]:
FilterOrSortOrCategorySum = FilterOrSortOrCategory['unique_events'].sum()

In [374]:
FilterOrSortOrCategorySum

85773

In [376]:
FilterOrSortOrCategory['in_percent'] = round(FilterOrSortOrCategory['unique_events'] * 100 / FilterOrSortOrCategorySum, 2)
FilterOrSortOrCategory

,sections,unique_events,in_percent
0,Категории,22894,26.69
1,Сортировка,19262,22.46
2,Фильтры,43617,50.85


Таким образом, мы видим, что число уникальных пользователей, которые использовали фильтры составляет 43617(50.85%), категории - 22894(26.69), сортировку - 19262()

4) Как часто люди пользуются выбором цены?

Тут есть два варианта, как ответить на этот вопрос. Зависит от того, что надо узнать. Если интересует анализ охвата аудитории, то следует смотреть на число уникальных пользователей, а если интересует активность пользователей или общая интенсивность использования функции, я бы анализировал total_events. Предположу, что Вас интересует охват.
Как я понял на сайте есть несколько фильтров, связанных с ценой. Поэтому я решил посмотреть все из них и просуммировать

In [381]:
unique_users_price_first = df.query("event_action == 'price_first'") \
                   .aggregate({'unique_events':'sum'})
unique_users_price_first

unique_events    6676
dtype: int64

In [383]:
unique_users_price_second = df.query("event_action == 'price_second'") \
                   .aggregate({'unique_events':'sum'})
unique_users_price_second

unique_events    1995
dtype: int64

In [385]:
unique_users_price_third = df.query("event_action == 'price_third'") \
                   .aggregate({'unique_events':'sum'})
unique_users_price_third

unique_events    595
dtype: int64

In [387]:
users_of_cost_filters = unique_users_price_first + unique_users_price_second + unique_users_price_third

In [389]:
users_of_cost_filters

unique_events    9266
dtype: int64

Фильтрами выбора цены пользовались 9266 человек. Но я хочу посмотреть сколько процентов это занимает от общего количества пользователей, которые используют фильтры

In [392]:
filter_value = FilterOrSortOrCategory.query("sections == 'Фильтры'")['unique_events'].values[0]
filter_value # Общее число пользователей, которые используют фильтры

43617

In [394]:
users_of_cost_filters_in_percent = round(users_of_cost_filters / filter_value * 100, 2)
users_of_cost_filters_in_percent

unique_events    21.24
dtype: float64

Фильтрами выбора цены пользовались 9266 человек, что составляет 21.24% от общего количества пользователей, которые используют фильтры.